In [ ]:
!wget "https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?"
!tar xvf "/content/genres.tar.gz?" -C "/content/"

In [ ]:
!pip install shutup
from google.colab import drive
drive.mount("/content/drive")

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from librosa import feature
from glob import glob
import pandas as pd
import numpy as np
import librosa
import csv
import scipy

import shutup
import os
import shutil
shutup.please()

Feature Extraction from Audio data and saving them in their respective .csv 📁 file 🔉

In [5]:
genre = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

In [6]:
def GenerateFeatures (path_to_file):
  audio = librosa.util.example_audio_file() 
  y,sr=librosa.load(path_to_file)

  y_harmonic, y_percussive = librosa.effects.hpss(y)
  tempo, beat_frames = librosa.beat.beat_track(y=y_harmonic, sr=sr)

  chroma=librosa.feature.chroma_cens(y=y_harmonic, sr=sr)
  mfccs = librosa.feature.mfcc(y=y_harmonic, sr=sr, n_mfcc=13)
  cent = librosa.feature.spectral_centroid(y=y, sr=sr)
  contrast=librosa.feature.spectral_contrast(y=y_harmonic,sr=sr)
  rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
  zrate=librosa.feature.zero_crossing_rate(y_harmonic)

  chroma_mean=np.mean(chroma,axis=1)
  chroma_std=np.std(chroma,axis=1)
  chroma_df=pd.DataFrame()
  for i in range(0,12):
      chroma_df['chroma_mean_'+str(i)]=chroma_mean[i]
  for i in range(0,12):
      chroma_df['chroma_std_'+str(i)]=chroma_mean[i]
  chroma_df.loc[0]=np.concatenate((chroma_mean,chroma_std),axis=0)

  mfccs_mean=np.mean(mfccs,axis=1)
  mfccs_std=np.std(mfccs,axis=1)
  mfccs_df=pd.DataFrame()
  for i in range(0,13):
      mfccs_df['mfccs_mean_'+str(i)]=mfccs_mean[i]
  for i in range(0,13):
     mfccs_df['mfccs_std_'+str(i)]=mfccs_mean[i]
  mfccs_df.loc[0]=np.concatenate((mfccs_mean,mfccs_std),axis=0)

  cent_mean=np.mean(cent)
  cent_std=np.std(cent)
  cent_skew=scipy.stats.skew(cent,axis=1)[0]
  contrast_mean=np.mean(contrast,axis=1)
  contrast_std=np.std(contrast,axis=1)  

  rolloff_mean=np.mean(rolloff)
  rolloff_std=np.std(rolloff)
  rolloff_skew=scipy.stats.skew(rolloff,axis=1)[0]

  spectral_df=pd.DataFrame()
  collist=['cent_mean','cent_std','cent_skew']
  for i in range(0,7):
      collist.append('contrast_mean_'+str(i))
  for i in range(0,7):
      collist.append('contrast_std_'+str(i))
  collist=collist+['rolloff_mean','rolloff_std','rolloff_skew']
  for c in collist:
      spectral_df[c]=0
  data=np.concatenate(([cent_mean,cent_std,cent_skew],contrast_mean,contrast_std,[rolloff_mean,rolloff_std,rolloff_std]),axis=0)
  spectral_df.loc[0]=data

  zrate_mean=np.mean(zrate)
  zrate_std=np.std(zrate)
  zrate_skew=scipy.stats.skew(zrate,axis=1)[0]

  zrate_df=pd.DataFrame()
  zrate_df['zrate_mean']=0
  zrate_df['zrate_std']=0
  zrate_df['zrate_skew']=0
  zrate_df.loc[0]=[zrate_mean,zrate_std,zrate_skew]

  beat_df=pd.DataFrame()
  beat_df['tempo']=tempo
  beat_df.loc[0]=tempo

  final_df=pd.concat((chroma_df,mfccs_df,spectral_df,zrate_df,beat_df),axis=1)

  return final_df

In [ ]:
GenerateFeatures('/content/genres/blues/blues.00000.au').columns

In [10]:
# dir = '/content/genres/'
# columns = ['chroma_mean_0', 'chroma_mean_1', 'chroma_mean_2', 'chroma_mean_3',
#        'chroma_mean_4', 'chroma_mean_5', 'chroma_mean_6', 'chroma_mean_7',
#        'chroma_mean_8', 'chroma_mean_9', 'chroma_mean_10', 'chroma_mean_11',
#        'chroma_std_0', 'chroma_std_1', 'chroma_std_2', 'chroma_std_3',
#        'chroma_std_4', 'chroma_std_5', 'chroma_std_6', 'chroma_std_7',
#        'chroma_std_8', 'chroma_std_9', 'chroma_std_10', 'chroma_std_11',
#        'mfccs_mean_0', 'mfccs_mean_1', 'mfccs_mean_2', 'mfccs_mean_3',
#        'mfccs_mean_4', 'mfccs_mean_5', 'mfccs_mean_6', 'mfccs_mean_7',
#        'mfccs_mean_8', 'mfccs_mean_9', 'mfccs_mean_10', 'mfccs_mean_11',
#        'mfccs_mean_12', 'mfccs_std_0', 'mfccs_std_1', 'mfccs_std_2',
#        'mfccs_std_3', 'mfccs_std_4', 'mfccs_std_5', 'mfccs_std_6',
#        'mfccs_std_7', 'mfccs_std_8', 'mfccs_std_9', 'mfccs_std_10',
#        'mfccs_std_11', 'mfccs_std_12', 'cent_mean', 'cent_std', 'cent_skew',
#        'contrast_mean_0', 'contrast_mean_1', 'contrast_mean_2',
#        'contrast_mean_3', 'contrast_mean_4', 'contrast_mean_5',
#        'contrast_mean_6', 'contrast_std_0', 'contrast_std_1', 'contrast_std_2',
#        'contrast_std_3', 'contrast_std_4', 'contrast_std_5', 'contrast_std_6',
#        'rolloff_mean', 'rolloff_std', 'rolloff_skew', 'zrate_mean',
#        'zrate_std', 'zrate_skew', 'tempo']

# for gen in genre:
#   print (f"Current gen: {gen}")
#   path = dir + gen

#   files = os.listdir(path+'/')
#   GenreFrame = pd.DataFrame(columns=columns)

#   for clip in files:
#     clip_path = dir + gen + '/' + clip

#     print(f'Extracting features from {clip_path}')
#     clipDataFrame = GenerateFeatures(clip_path)
#     clipDataFrame['label'] = gen
  
#     GenreFrame = pd.concat((GenreFrame, clipDataFrame), axis=0)

#   GenreFrame.to_csv('/content/' + gen + '.csv')
#   print(f'Label: {gen} saved at /content/{gen}.csv\n\n\n\n')
#   shutil.copy (f"/content/{gen}.csv", f"/content/drive/MyDrive/IITK DL datasets/{gen}.csv")


Reading .csv file and adding a new column type ✔

In [11]:
blues = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/blues.csv")
classical = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/classical.csv")
country = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/country.csv")
disco = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/disco.csv")
hiphop = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/hiphop.csv")
jazz = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/jazz.csv")
metal = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/metal.csv")
pop = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/pop.csv")
reggae = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/reggae.csv")
rock = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/rock.csv")

blues.drop(["Unnamed: 0"], inplace=True, axis=1)
classical.drop(["Unnamed: 0"], inplace=True, axis=1)
country.drop(["Unnamed: 0"], inplace=True, axis=1)
disco.drop(["Unnamed: 0"], inplace=True, axis=1)
hiphop.drop(["Unnamed: 0"], inplace=True, axis=1)
jazz.drop(["Unnamed: 0"], inplace=True, axis=1)
metal.drop(["Unnamed: 0"], inplace=True, axis=1)
pop.drop(["Unnamed: 0"], inplace=True, axis=1)
reggae.drop(["Unnamed: 0"], inplace=True, axis=1)
rock.drop(["Unnamed: 0"], inplace=True, axis=1)

blues['label'] = 'blues'
classical['label'] = 'classical'
country['label'] = 'country'
disco['label'] = 'disco'
hiphop['label'] = 'hiphop'
jazz['label'] = 'jazz'
metal['label'] = 'metal'
pop['label'] = 'pop'
reggae['label'] = 'reggae'
rock['label'] = 'rock'

Creating The Main Dataframe for training our Model 🚩

In [12]:
final_df = pd.DataFrame(columns = rock.columns)

In [13]:
genres = [blues, 
          classical, 
          country,
          hiphop, 
          jazz,
          metal, 
          pop,
          reggae,
          disco,
          rock]

final_df = pd.concat(genres)

In [ ]:
final_df.columns

Encoding labels of Target column ✈

In [33]:
final_df.sample(20)

,chroma_mean_0,chroma_mean_1,chroma_mean_2,chroma_mean_3,chroma_mean_4,chroma_mean_5,chroma_mean_6,chroma_mean_7,chroma_mean_8,chroma_mean_9,...,contrast_std_5,contrast_std_6,rolloff_mean,rolloff_std,rolloff_skew,zrate_mean,zrate_std,zrate_skew,tempo,label
15,0.173040,0.125036,0.381547,0.081684,0.255975,0.044286,0.149351,0.485594,0.293587,0.233636,...,2.151185,1.484845,7959.674426,1267.143679,1267.143679,0.077210,0.028730,0.804365,135.999178,pop
31,0.176231,0.256152,0.454807,0.287205,0.142508,0.164120,0.169230,0.137201,0.065159,0.126814,...,6.067940,2.088454,1013.675953,274.697797,274.697797,0.046949,0.014731,0.777574,258.398438,classical
80,0.366663,0.122814,0.268694,0.314878,0.249233,0.274389,0.157577,0.243907,0.136973,0.171319,...,2.749511,1.706806,4516.110562,842.477574,842.477574,0.062749,0.023487,1.010985,107.666016,disco
90,0.016943,0.101259,0.116911,0.547772,0.134592,0.188521,0.368483,0.227065,0.412011,0.085392,...,2.163967,2.461817,3285.628727,703.521871,703.521871,0.032130,0.022221,1.993907,151.999081,metal
65,0.168811,0.236189,0.161380,0.168440,0.346705,0.148578,0.141563,0.127635,0.200722,0.276658,...,2.520092,2.022049,6974.842640,1846.422299,1846.422299,0.055712,0.050471,1.734417,86.132812,reggae
27,0.100054,0.101207,0.022796,0.064754,0.521932,0.127752,0.164793,0.135781,0.156913,0.597486,...,2.811296,2.306018,3242.912048,531.120193,531.120193,0.063319,0.023750,0.465577,151.999081,rock
2,0.180265,0.095818,0.349376,0.203895,0.175580,0.231949,0.291783,0.494568,0.281742,0.193693,...,3.309823,1.595846,5302.980102,1612.431068,1612.431068,0.102110,0.029770,0.407494,117.453835,rock
31,0.149201,0.076278,0.176088,0.125718,0.297446,0.179517,0.316198,0.279485,0.190335,0.178577,...,2.211744,1.819928,4949.930496,876.562827,876.562827,0.092888,0.031362,0.566867,112.347147,metal
28,0.155435,0.160356,0.094265,0.140142,0.202913,0.268784,0.265478,0.256553,0.309287,0.325848,...,3.820788,4.352897,3840.287735,2114.343241,2114.343241,0.048349,0.032443,1.094712,103.359375,reggae
74,0.365848,0.045748,0.065096,0.038490,0.267070,0.524175,0.068797,0.176040,0.087810,0.161682,...,4.539179,3.274140,2347.310658,1210.802248,1210.802248,0.028383,0.018429,1.467272,71.777344,country


In [15]:
convertor = LabelEncoder()
class_list = final_df.iloc[:,-1]
y = convertor.fit_transform(class_list)

Fitting Rest of the columns in StanderdScaler (Using fit_transform for that)

In [16]:
fit = StandardScaler()
X = fit.fit_transform(np.array(final_df.iloc[:,:-1], dtype = float))

In [17]:
X

array([[ 0.82833167, -1.33495166, -1.28353263, ..., -1.47272182,
         0.40958727, -0.51955897],
       [ 0.76552063, -1.29965731, -0.58771025, ...,  0.05443765,
        -0.66389224,  2.18518953],
       [ 0.13282168, -0.8275167 , -0.73332123, ..., -0.24657498,
         1.09981381, -1.91463978],
       ...,
       [ 0.71974078, -0.6994479 , -0.80776934, ..., -0.42292   ,
        -0.77951094,  0.02139073],
       [-1.90772386,  1.30226094, -1.81263156, ...,  0.18421847,
         0.79699784, -0.35492211],
       [-0.2659568 , -1.40462779,  0.03128313, ..., -0.12075206,
         0.27557264, -1.63327894]])

Splitting Our Dataset for Training (x_train & y_train) and Testing (y_train & y_test)

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y ,test_size=0.33)

Creating TrainModel Function for training our RNN model.

In [19]:
def trainModel (model, epochs, optimizer, verbose):
  batch_size = 64

  model.compile(optimizer,
                loss='sparse_categorical_crossentropy',
                metrics='accuracy')

  return model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 epochs=epochs, batch_size=batch_size, verbose=verbose)

Our Sequential Model ⚖

In [ ]:
model = keras.models.Sequential([
                             keras.layers.Dense(512, activation='relu', input_shape=(x_train.shape[1],)),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(256, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(128, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(64, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Flatten(),
                             keras.layers.Dense(10, activation='softmax')
                          ])
print(model.summary())
model_history = trainModel(model=model, epochs=60, optimizer='adam', verbose=2)

In [21]:
test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=64)
print (f"\nThe test loss is : {round(test_loss, 2)}")
print(f"The best test accuracy is : {round(test_acc*100, 2)}%")

6/6 [==============================] - 0s 3ms/step - loss: 1.6283 - accuracy: 0.7091

The test loss is : 1.63
The best test accuracy is : 70.91%


In [ ]:
model.summary()

In [41]:
from sklearn.preprocessing import StandardScaler as ss

def makeAudio_to_Data (path):
  newDataframe = GenerateFeatures (path)
  fit = ss()
  numpyArray = fit.fit_transform(np.array(newDataframe, dtype=float))

  return numpyArray, newDataframe

In [ ]:
frame

In [42]:
frame, data = makeAudio_to_Data ("/content/genres/pop/pop.00000.au")

In [ ]:
arr = model.predict(frame)
lis = arr.tolist()
print(genre[lis[0].index(max(lis[0]))].capitalize())
print(lis[0])

In [ ]:
final_df.columns

In [ ]:
frame

In [39]:
data.sample()

,chroma_mean_0,chroma_mean_1,chroma_mean_2,chroma_mean_3,chroma_mean_4,chroma_mean_5,chroma_mean_6,chroma_mean_7,chroma_mean_8,chroma_mean_9,...,contrast_std_4,contrast_std_5,contrast_std_6,rolloff_mean,rolloff_std,rolloff_skew,zrate_mean,zrate_std,zrate_skew,tempo
0,0.267179,0.240497,0.293637,0.191024,0.210611,0.230965,0.205257,0.336119,0.165912,0.163913,...,4.054335,3.124785,2.1646,7952.938014,1643.261345,1643.261345,0.075698,0.061506,2.504689,89.102909


In [ ]:
shutil.copy("", "")